# File with both models

In [1]:
from keras.saving import register_keras_serializable

# Binary Model

In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score


# Load IMDB data
max_features = 5000  # Vocabulary size
maxlen = 500         # Maximum sequence length

(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)

# Pad sequences
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

# Build the model
model_bin = keras.Sequential([
 layers.Embedding(input_dim=max_features, output_dim=128, input_length=maxlen),
    layers.Conv1D(filters=128, kernel_size=5, activation='relu', padding='same'),
    layers.GlobalMaxPooling1D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])


# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model_bin.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history = model_bin.fit(x_train, y_train,
                    epochs=20, # Added 10 epochs
                    batch_size=64,
                    validation_split=0.2,
                    callbacks=[early_stopping])

# Evaluate the model
test_loss, test_acc = model_bin.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

# Plot training & validation accuracy and loss
history_dict = history.history
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']
epochs = range(1, len(acc) + 1)


#Soruce: https://bagheri365.github.io/blog/Sentiment-Analysis-of-IMDB-Movie-Reviews-using-Convolutional-Neural-Network-%28CNN%29-with-Hyperparameters-Tuning/?utm_source=chatgpt.com


## SCORES

# Predict probabilities
y_pred_prob = model_bin.predict(x_test)

# Convert probabilities to binary predictions
y_pred = (y_pred_prob > 0.5).astype("int32")

# Compute F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 Score: {f1:.4f}')

c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 17s 50ms/step - accuracy: 0.5234 - loss: 0.6918 - val_accuracy: 0.6834 - val_loss: 0.6648
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 71ms/step - accuracy: 0.6736 - loss: 0.6368 - val_accuracy: 0.7264 - val_loss: 0.5541
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 72ms/step - accuracy: 0.7570 - loss: 0.5184 - val_accuracy: 0.8054 - val_loss: 0.4425
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 25s 80ms/step - accuracy: 0.8252 - loss: 0.4053 - val_accuracy: 0.8472 - val_loss: 0.3682
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 49s 155ms/step - accuracy: 0.8642 - loss: 0.3319 - val_accuracy: 0.8650 - val_loss: 0.3209
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 64ms/step - accuracy: 0.8997 - loss: 0.2652 - val_accuracy: 0.8808 - val_loss: 0.2907
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 74ms/step - accuracy: 0.9194 - loss: 0.2163 - val_accuracy: 0.8900 - val_loss: 0.2732
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 37s 118ms/step - accuracy: 0.9458 - loss: 0.1697 

# Multiclass Model - Transformers

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, Conv1D
from tensorflow.keras.layers import LSTM, Bidirectional, Dropout, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import random
import os
from tensorflow.keras import layers

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


In [4]:
import pandas as pd
#Download the goemotions dataset
!gsutil cp -r gs://gresearch/goemotions/data/full_dataset/ .

# Load the CSV files
df1 = pd.read_csv("full_dataset/goemotions_1.csv")
df2 = pd.read_csv("full_dataset/goemotions_2.csv")
df3 = pd.read_csv("full_dataset/goemotions_3.csv")

# Concatenate them into one dataframe
df = pd.concat([df1, df2, df3], ignore_index=True)

# To reduce the runtime we use a random sample of 50.000 datapoints
df_small = df.sample(n=50000, random_state=42)

Copying gs://gresearch/goemotions/data/full_dataset/goemotions_1.csv...
/ [0 files][    0.0 B/ 13.5 MiB]                                                
/ [0 files][320.0 KiB/ 13.5 MiB]                                                
-
- [1 files][ 13.5 MiB/ 13.5 MiB]                                                
Copying gs://gresearch/goemotions/data/full_dataset/goemotions_2.csv...
- [1 files][ 13.5 MiB/ 27.0 MiB]                                                
\
\ [1 files][ 17.3 MiB/ 27.0 MiB]                                                
\ [2 files][ 27.0 MiB/ 27.0 MiB]                                                
Copying gs://gresearch/goemotions/data/full_dataset/goemotions_3.csv...
\ [2 files][ 27.0 MiB/ 40.8 MiB]                                                
\ [2 files][ 27.4 MiB/ 40.8 MiB]                                                
|
/
/ [3 files][ 40.8 MiB/ 40.8 MiB]                                                

Operation completed over 3 objects/40.8 MiB.  

In [5]:
# Identify emotion columns
emotion_columns = df_small.columns[df_small.columns.get_loc('example_very_unclear') + 1:].tolist()
print(f"Number of emotion labels: {len(emotion_columns)}")
print(f"Emotion labels: {emotion_columns}")

# Data Preprocessing
# Define features and target
X = df_small['text']
y = df_small[emotion_columns].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text data
max_words = 10000  # Max vocabulary size
max_len = 100  # Max sequence length

tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>') #any word that exceeds the worklimit will be set by that oov token.
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure uniform length
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

print(f"Vocabulary Size: {len(tokenizer.word_index)}")
print(f"Padded Training Data Shape: {X_train_pad.shape}")

Number of emotion labels: 28
Emotion labels: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']
Vocabulary Size: 23655
Padded Training Data Shape: (40000, 100)


In [6]:
class_counts = np.sum(y_train, axis=0)
total_samples = len(y_train)
class_weights = {}
for i in range(len(emotion_columns)):
    # Calculate weight as inverse of class frequency, normalized
    weight = total_samples / (len(emotion_columns) * class_counts[i])
    class_weights[i] = min(weight, 10.0)  # Cap the weight to prevent extreme values

print("Class weights:", class_weights)

Class weights: {0: np.float64(0.4293872643737387), 1: np.float64(0.7963051441312311), 2: np.float64(0.934317481080071), 3: np.float64(0.5597850425436632), 4: np.float64(0.4411894467484338), 5: np.float64(1.3190871916633689), 6: np.float64(1.0299721907508497), 7: np.float64(0.7785130400934216), 8: np.float64(1.9786307874950535), 9: np.float64(0.9070294784580499), 10: np.float64(0.6580246101204185), 11: np.float64(1.3950892857142858), 12: np.float64(2.886002886002886), 13: np.float64(1.3605442176870748), 14: np.float64(2.4461839530332683), 15: np.float64(0.6586313640255549), 16: 10.0, 17: np.float64(0.9523809523809523), 18: np.float64(0.9404683532399135), 19: np.float64(4.214075010535187), 20: np.float64(0.833472245374229), 21: np.float64(5.2521008403361344), 22: np.float64(0.8689607229753216), 23: np.float64(5.668934240362812), 24: np.float64(3.007518796992481), 25: np.float64(1.1690437222352117), 26: np.float64(1.411631846414455), 27: np.float64(0.13487268018990073)}


# Defining Loss function

In [7]:
@register_keras_serializable()
def focal_loss_with_penalty(y_true, y_pred, gamma=2.0, alpha=0.25, epsilon=1e-7):
    # Focal loss for multi-label classification
    y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
    cross_entropy = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)

    p_t = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
    focal_loss = alpha * tf.pow(1 - p_t, gamma) * cross_entropy

    # Add penalty for all-zero predictions
    sum_pred = tf.reduce_sum(y_pred, axis=1)
    all_zero_penalty = 5.0 * tf.exp(-sum_pred)  # Penalty increases as sum approaches zero

    return tf.reduce_mean(focal_loss) + tf.reduce_mean(all_zero_penalty)

# Define F1 score

In [8]:

"""
Custom F1 score metric for Keras.
Implements F1 score calculation directly in TensorFlow for use during training.
"""
@register_keras_serializable()
def f1_metric(y_true, y_pred):
    y_pred_binary = tf.cast(tf.greater(y_pred, 0.5), tf.float32)

    true_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred_binary, 1)), tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(y_pred_binary, 1)), tf.float32))
    false_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred_binary, 0)), tf.float32))

    precision = true_positives / (true_positives + false_positives + tf.keras.backend.epsilon())
    recall = true_positives / (true_positives + false_negatives + tf.keras.backend.epsilon())

    f1 = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())
    return f1

# Prediction Function


In [9]:

"""
Threshold-based prediction function that ensures at least one emotion is predicted.
Addresses the requirement to prevent all-zero predictions.
Based on the principles discussed in Lipton et al., 2014 [5] with our adaptation
for multi-label classification.
"""
def predict_with_threshold(model, X, threshold=0.5):
    y_pred_proba = model.predict(X)
    y_pred = (y_pred_proba >= threshold).astype(int)

    # If any row has all zeros, set the highest probability class to 1
    zero_rows = np.where(np.sum(y_pred, axis=1) == 0)[0]
    for row in zero_rows:
        max_prob_idx = np.argmax(y_pred_proba[row])
        y_pred[row, max_prob_idx] = 1

    return y_pred


# Transformer model

In [10]:
@register_keras_serializable()
class LearnablePositionalEncoding(layers.Layer):
    def __init__(self, maxlen, embedding_dim, **kwargs):
        super().__init__(**kwargs)
        self.pos_embedding = self.add_weight(
            shape=(maxlen, embedding_dim),
            initializer='random_normal',
            trainable=True,
            name="learnable_pos_embedding"
        )

    def call(self, x):
        # x shape: (batch_size, sequence_length, embedding_dim)
        seq_len = tf.shape(x)[1]
        return x + self.pos_embedding[tf.newaxis, :seq_len, :]

In [11]:
#model with learnable positional encoding
def build_transformer_model(vocab_size, embedding_dim=100, num_heads=8, ff_dim=128, dropout_rate=0.1):
    inputs = Input(shape=(max_len,))

    # Embedding + positional encoding
    x = Embedding(vocab_size, embedding_dim)(inputs)
    x = LearnablePositionalEncoding(maxlen=max_len, embedding_dim=embedding_dim)(x)

    # Transformer block
    attention_output = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embedding_dim // num_heads
    )(x, x)

    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention_output + x)

    # Feed Forward
    ff = Dense(ff_dim, activation='relu')(x)
    ff = Dense(embedding_dim)(ff)
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x + ff)

    # Classification head
    x = GlobalMaxPooling1D()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(len(emotion_columns), activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss=focal_loss_with_penalty,
        metrics=['accuracy', f1_metric]
    )
    return model

# callbacks

In [12]:
# Set up callbacks
callbacks = [
    EarlyStopping(monitor='val_f1_metric', patience=4, mode='max', restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_f1_metric', mode='max', save_best_only=True)
]

In [13]:
def train_and_evaluate(model, model_name):
    print(f"\n=== Training {model_name} ===")

    history = model.fit(
        X_train_pad, y_train,
        epochs=10,
        batch_size=32,
        validation_split=0.2,
        callbacks=callbacks,
        class_weight=class_weights,
        verbose=1
    )

    # Evaluate on test set
    y_pred = predict_with_threshold(model, X_test_pad, threshold=0.5)

    # Calculate F1 scores
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    micro_f1 = f1_score(y_test, y_pred, average='micro')
    weighted_f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"\n{model_name} Results:")
    print(f"Macro F1 Score: {macro_f1:.4f}")
    print(f"Micro F1 Score: {micro_f1:.4f}")
    print(f"Weighted F1 Score: {weighted_f1:.4f}")

    # Check if any all-zero predictions remain
    zero_preds = (np.sum(y_pred, axis=1) == 0).sum()
    print(f"Texts with no emotion predictions: {zero_preds} ({zero_preds/len(y_test)*100:.2f}%)")


    return model, macro_f1, micro_f1, weighted_f1, y_pred


In [14]:
# Initialize and train Transformer model
print("\nBuilding Transformer model...")
transformer_model = build_transformer_model(vocab_size=min(len(tokenizer.word_index) + 1, max_words))
transformer_model.summary()
transformer_model, transformer_macro_f1, transformer_micro_f1, transformer_weighted_f1, transformer_preds = train_and_evaluate(transformer_model, "Transformer")


Building Transformer model...



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 100, 100)  │  1,000,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ learnable_position… │ (None, 100, 100)  │     10,000 │ embedding_1[0][0] │
│ (LearnablePosition… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 100, 100)  │     38,788 │ learnable_positi… │
│ (MultiHeadAttentio… │                   │            │ learnable_positi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 100, 100)  │          0 │ multi_head_atten… │
│                     │                   │            │ learnable_positi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 100, 100)  │        200 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 100, 128)  │     12,928 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 100, 100)  │     12,900 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 100, 100)  │          0 │ layer_normalizat… │
│                     │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 100, 100)  │        200 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 100)       │          0 │ layer_normalizat… │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 100)       │          0 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │     25,856 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 256)       │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 28)        │      7,196 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,108,068 (4.23 MB)

 Trainable params: 1,108,068 (4.23 MB)

 Non-trainable params: 0 (0.00 B)


=== Training Transformer ===
Epoch 1/10
 999/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.2162 - f1_metric: 0.0487 - loss: 0.0131

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 54s 46ms/step - accuracy: 0.2163 - f1_metric: 0.0487 - loss: 0.0131 - val_accuracy: 0.3571 - val_f1_metric: 0.2070 - val_loss: 0.0135
Epoch 2/10
 999/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.3597 - f1_metric: 0.1883 - loss: 0.0101

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - accuracy: 0.3597 - f1_metric: 0.1883 - loss: 0.0101 - val_accuracy: 0.3884 - val_f1_metric: 0.3182 - val_loss: 0.0130
Epoch 3/10
 999/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.3872 - f1_metric: 0.2902 - loss: 0.0097

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 27s 27ms/step - accuracy: 0.3872 - f1_metric: 0.2902 - loss: 0.0097 - val_accuracy: 0.3915 - val_f1_metric: 0.3369 - val_loss: 0.0130
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 30s 30ms/step - accuracy: 0.4155 - f1_metric: 0.3483 - loss: 0.0094 - val_accuracy: 0.3814 - val_f1_metric: 0.3340 - val_loss: 0.0130
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 34s 23ms/step - accuracy: 0.4497 - f1_metric: 0.3996 - loss: 0.0092 - val_accuracy: 0.3639 - val_f1_metric: 0.3343 - val_loss: 0.0132
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 34s 34ms/step - accuracy: 0.4900 - f1_metric: 0.4547 - loss: 0.0089 - val_accuracy: 0.3549 - val_f1_metric: 0.3350 - val_loss: 0.0134
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 43s 42ms/step - accuracy: 0.5324 - f1_metric: 0.5127 - loss: 0.0086 - val_accuracy: 0.3417 - val_f1_metric: 0.3339 - val_loss: 0.0137
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step

Transformer Results:
Macro F1 Score: 0.2307
Micro F1 Score: 0.3911
Weighted F1 Score: 0.3

# optimize threshold

In [15]:
def optimize_threshold(model, X, y_true, thresholds=None):
    if thresholds is None:
        thresholds = np.arange(0.2, 0.8, 0.05)

    results = []

    for threshold in thresholds:
        y_pred = predict_with_threshold(model, X, threshold=threshold)
        micro_f1 = f1_score(y_true, y_pred, average='micro')
        results.append((threshold, micro_f1))
        print(f"Threshold: {threshold:.2f}, Micro F1: {micro_f1:.4f}")

    best_threshold, best_f1 = max(results, key=lambda x: x[1])
    print(f"\nBest threshold: {best_threshold:.2f}")
    print(f"Best F1 score: {best_f1:.4f}")

    return best_threshold

"""
Threshold Optimization and Final Evaluation
"""
# Optimize threshold for best model
print("\nOptimizing threshold for best model...")
best_threshold = optimize_threshold(transformer_model, X_test_pad, y_test)


Optimizing threshold for best model...


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
Threshold: 0.20, Micro F1: 0.0812
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step
Threshold: 0.25, Micro F1: 0.0976
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step
Threshold: 0.30, Micro F1: 0.1967
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step
Threshold: 0.35, Micro F1: 0.3233
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
Threshold: 0.40, Micro F1: 0.3926
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
Threshold: 0.45, Micro F1: 0.3990
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
Threshold: 0.50, Micro F1: 0.3911
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
Threshold: 0.55, Micro F1: 0.3868
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
Threshold: 0.60, Micro F1: 0.3860
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
Threshold: 0.65, Micro F1: 0.3857
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
Threshold: 0.70, Micro F1: 0.3857
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
Threshold: 0.75, Micro F1: 0.3857
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
Threshold: 0.80, Micro F1: 0.3857

Best threshold: 0.45
B

# Saving the models

In [18]:
from keras.datasets import imdb

# Load the original word index
word_index = imdb.get_word_index()

# Shift indices by 3 to reserve special tokens
word_index = {word: (index + 3) for word, index in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

import re


transformer_model.save("emotion_model.keras")
model_bin.save("sentiment_model.keras")

import pickle

# Save tokenizer
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

# Save word_index and emotion_columns if used explicitly
with open("assets.pkl", "wb") as f:
    pickle.dump({'word_index': word_index, 'emotion_columns': emotion_columns, 'best_threshold': best_threshold}, f)

# Audio Streaming 

In [20]:
import whisper
import numpy as np
import queue
import threading
import pyaudio
import time
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Initialize the Whisper model
whisper_model = whisper.load_model("base") 

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 3

p = pyaudio.PyAudio()
audio_queue = queue.Queue()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)


def process_audio():
    while True:
        audio_data = audio_queue.get()
        # to float32 in range [-1,1]
        audio_np = np.frombuffer(audio_data, dtype=np.int16).astype(np.float32) / 32768.0

        # Whisper transcription
        result = whisper_model.transcribe(audio_np, fp16=False)
        text = result["text"].strip()
        if not text:
            continue

        print(f"Transcribed: {text}")

        # Sentiment
        sentiment = model_bin(text)
        print(f"Sentiment: {sentiment}")

        # Emotions
        emotions = transformer_model([text])[0]
        print(f"Emotions: {emotions}")

# start background thread
threading.Thread(target=process_audio, daemon=True).start()


try:
    print("* Recording started — press Ctrl+C to stop")
    while True:
        frames = []
        for _ in range(int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK, exception_on_overflow=False)
            frames.append(data)
        audio_queue.put(b"".join(frames))

except KeyboardInterrupt:
    print("\n* Recording stopped")

finally:
    stream.stop_stream()
    stream.close()
    p.terminate()

* Recording started — press Ctrl+C to stop


Exception in thread Thread-15 (process_audio):
Traceback (most recent call last):
  File "c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\franc\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\franc\AppData\Local\Temp\ipykernel_16572\1715734442.py", line 43, in process_audio
  File "c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\layer.py", line 814, in __call__
    raise ValueError(
ValueError: Only input tensors may be passed 

Transcribed: Okay. Okay.

* Recording stopped
